## Langevin Diffusion based MCMC

Here, we sample from some density on parameter $\theta$, such that it can be represented as exponential of some U($\theta$) also called the potential function.Let's take an example of some independent events and hence the overall probability can be represented as products of all such events. So, in such scenerio, we can assume U($\theta$) as sum of such different independent events because $\pi(\theta)$ is product of independent events.<br>
So, from Bayesian Likelihood,<br>
If $\pi(\theta) = p(\theta) \prod_{i=1}^N f(y_i|\theta)$<br>
Hence, $U(\theta) = \sum_{i=1}^N U_i(\theta)$<br>
If i want to sample from $\pi(\theta)$ i can simulate from distribution have $\pi$ as its stationary distribution and by simulation i think of using MCMC techniques.<br>
Langevin differential equation is one such equation with $\pi$ as its stationary distribution.<br>
d($\theta(t)$) = $-\frac{1}{2}\nabla U(\theta(t))dt+dB_t$<br>
If time-period are small, the dynamics can be approximated by <br>
$\theta(t+h)-\theta(t) \approx -\frac{h}{2}\nabla U(\theta(t))dt+\sqrt h Z$, where Z is simply some Gaussian Noise<br>
If i start with some $\theta _O$ then $\theta _K$ can be teh state at some given time Kh and decreasing the h, will increase accuracy<br>
## Approximate MCMC from LD

So, if we can simulate Langevin Diffusion then we get pretty good result but using the actual equation is quite hard so we generally use the approximation given above. Based on above observations, MALA algo uses such euler approximation to define MH proposal and then goes through Accept reject based on the ratio obtained.Similar, but simpler algo is ULA, which doesn't include the Accept Reject.This produces a biased result but takes less number of iterations but that didn't help much because we have to calculate more number of gradient term which are computationally as expensive as Accept reject.Advantaeg of ULA is that it performs better even in poor initialization which is not a case in MALA.<br>
A solution to computation of $U(\theta)$ is computation of $ \sum_{i=1}^N U_i$ instead we can calculate it for some random subsample of size n which is much less than N and that to without replacemnet from {1 to N}<br>
$\hat{\nabla}U(\theta)^{n} = \frac{N}{n}\sum_{subset}\nabla U_i(\theta)$<br>
SGLD Algo can be summarized as:-
1. Take a random subset and estimate gradient as subset sum for $\hat{\nabla}U(\theta)^{n}$
2. Draw noise $E_k$ from a gaussian distribution with zero mean and variance as $h_kI$
3. Use the Euler approximation to LG, $\theta_{k+1} \leftarrow \theta_{k}-\frac{h_k}{2} \hat{\nabla}U(\theta)^{n} + E_k$<br>
Thus, SGLD gives advantage over MALA or ULA if there is fixed computation.SGLD has sometimes more efficient than SGD, only difference is we have noise in SGLD and decreasing step size in SGD.<br>

## Estimating Gradient
What we see is if we estimate gradient better we will get a better estimate.We can use Control variates technique to reduce variances.We take $u_i(\theta)$ as functions whose sum is already known for all $\theta$.Let use $ u_i=\nabla U_i(\hat{\theta})$ for some initial distribution $\hat{\theta}$.<br>
So $ \sum_{i=1}^N \nabla U_i(\theta) = \sum_{i=1}^N u_i(\theta)+\sum_{i=1}^N(\nabla U_i(\theta)-u_i(\theta))$. Using the similar subset sum technique and above set of $u_i$<br>
$ \hat{\nabla} U_{cv}(\theta) = \sum_{i=1}^N u_i(\theta)+\frac{N}{n}\sum_{subset}(\nabla U_i(\theta)-\nabla U_i(\hat{\theta}))$.<br>
Now, if we compare variance in cv and previous approach, they cv depends on distance between initial theta's>hence, if we sample from distance of $N^\frac{-1}{2}$ then we can reduce from $O(\frac{N^2}{n})$ to $O(\frac{N}{n})$<br>
Another way to reduce variance is to use weighted estimator<br>
$ \hat{\nabla} U_{w}(\theta) = \sum_{subset} \nabla \frac{U_i(\theta)}{w_i}$, using larger $w_i$ for grad values further from mean.

## General Framework for SGMCMC 
There can be many more Scholastic Differential equation. A general one for a state $\zeta = (\theta, \rho)$, the DE is<br>
d $\zeta = \frac{1}{2}b(\zeta)dt + \sqrt D(\zeta)dB_t$<br>
For stationary distribution to be proportional to $e^{-H(\zeta)}$ and for some Q($\zeta$), skew-symmetric curl matrix, then choice should be<br>
b($\zeta$) = -\[D($\zeta$)+Q($\zeta$)]$\nabla H(\zeta)$+$\Gamma(\zeta)$ and $\Gamma _i(\zeta) = \sum_{j=1}^d \frac{\partial}{\partial \zeta_{j}}(D_{ij}(\zeta)+Q_{ij}(\zeta))$<br>
Employing similiar Euler discretization we make a similiar equation as LD.<br>
To have a stable MCMC algo we need D($\zeta _t$)-h $ \hat{B}( \zeta _t)$ to be positive semi-definite<br>
D($\zeta$) and Q($\zeta$) affects convergence.If Diffusion matrix is large, the sample escapes and sampler value increases accuracy.<br>
Two most commonly used are SGLD and SGHMC, where we have a velocity component as well.